In [71]:
import os
import jiwer

def calculate_wer(reference, hypothesis):
    """
    Calculate the Word Error Rate (WER) between reference and hypothesis texts.
    """
    transformation = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.Strip(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(word_delimiter=" ")
    ])
    return jiwer.wer(reference, hypothesis, truth_transform=transformation, hypothesis_transform=transformation)

echomind_output_dir = "C:\\Users\\gdhir\\Downloads\\EchoMind-main\\EchoMind-main\\output"
timit_tests_dir = "C:\\Users\\gdhir\\Downloads\\TimitDataset\\TimitDataset\\tests"
total_wer = 0
count = 0
for root, dirs, files in os.walk(echomind_output_dir):
    if 'DR' in os.path.basename(root) and 'DR' in os.path.basename(os.path.dirname(root)):
        for file in files:
            if file == 'combined.txt':
                combined_file_path = os.path.join(root, file)
                dr_folder = os.path.basename(root)
                reference_file_path = os.path.join(timit_tests_dir, dr_folder, 'text.txt')

                if os.path.exists(combined_file_path) and os.path.exists(reference_file_path):
                    with open(combined_file_path, 'r', encoding='utf-8') as c_file, open(reference_file_path, 'r', encoding='utf-8') as r_file:
                        combined_text = c_file.read()
                        reference_text = r_file.read()

                    wer = calculate_wer(reference_text, combined_text)
                    total_wer += wer
                    count += 1
                else:
                    print(f"Missing files for directory: {dr_folder}")

if count > 0:
    average_wer = total_wer / count
    print(f"Average WER for all transcripts: {average_wer * 100:.2f}%")
else:
    print("No transcripts processed.")

Average WER for all transcripts: 12.31%


In [70]:
import os
from collections import Counter
from difflib import SequenceMatcher
from spellchecker import SpellChecker
from transformers import pipeline

def align_transcripts(transcripts):
    base_transcript = transcripts[0]
    aligned_transcripts = [base_transcript.split()]
    for transcript in transcripts[1:]:
        match = SequenceMatcher(None, base_transcript, transcript)
        aligned_transcript = []
        for opcode in match.get_opcodes():
            if opcode[0] == 'equal':
                aligned_transcript.extend(transcript[opcode[3]:opcode[4]].split())
            elif opcode[0] == 'replace' or opcode[0] == 'insert':
                aligned_transcript.extend(['<blank>'] * (opcode[2] - opcode[1]))
        aligned_transcripts.append(aligned_transcript)
    return aligned_transcripts

def spell_correct_transcript(transcript):
    spell = SpellChecker()
    corrected_words = []
    for word in transcript.split():
        corrected_word = spell.correction(word)
        if corrected_word:
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

def vote_on_words(aligned_transcripts):
    final_transcript = []
    for word_tuple in zip(*aligned_transcripts):
        word_counts = Counter(word_tuple)
        if word_counts.most_common(1)[0][0] != '<blank>':
            final_transcript.append(word_counts.most_common(1)[0][0])
        else:
            final_transcript.append(word_counts.most_common(2)[1][0])
    return ' '.join(final_transcript)

def combine_transcripts(transcripts):
    aligned_transcripts = align_transcripts(transcripts)
    combined_transcript = vote_on_words(aligned_transcripts)
    combined_transcript = combined_transcript.replace('<blank> ', '')
    spell_corrected_transcript = spell_correct_transcript(combined_transcript)
    return spell_corrected_transcript

def read_transcript(file_path):
    with open(file_path, 'r') as file:
        return file.read()

root_dir = "C:\\Users\\gdhir\\Downloads\\EchoMind-main\\EchoMind-main\\output"

for root, dirs, files in os.walk(root_dir):
    if 'DR' in root:  
        print(f"Processing directory: {root}")
        for sub_dir in dirs:
            if sub_dir.startswith('DR'):
                transcripts = []
                transcript_files = ['Wav2vec_transcipt.txt', 'vosk_transcipt.txt', 'Whisper_transcipt.txt']
                for t_file in transcript_files:
                    file_path = os.path.join(root, sub_dir, t_file)
                    if os.path.exists(file_path):
                        transcripts.append(read_transcript(file_path))
                    else:
                        print(f"Missing transcript file: {file_path}")

                if len(transcripts) == 3:  
                    combined_transcript = combine_transcripts(transcripts)
                    combined_file_path = os.path.join(root, sub_dir, 'combined.txt')
                    with open(combined_file_path, 'w', encoding='utf-8') as file:
                        file.write(combined_transcript)
                    print(f"Combined transcript written to {combined_file_path}")
                else:
                    print(f"Skipping directory {sub_dir} due to missing transcripts")

print("Transcripts combined successfully.")

Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SA1
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SA1\DR1FAKS0SA1\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SA1\DR1FAKS0SA1
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SA2
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SA2\DR1FAKS0SA2\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SA2\DR1FAKS0SA2
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SI1573
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SI1573\DR1FAKS0SI1573\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FAKS0SI1573\DR1FAKS0SI1573
P

Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FELC0SX216\DR1FELC0SX216\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FELC0SX216\DR1FELC0SX216
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FELC0SX306
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FELC0SX306\DR1FELC0SX306\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FELC0SX306\DR1FELC0SX306
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FELC0SX36
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FELC0SX36\DR1FELC0SX36\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FELC0SX36\DR1FELC0SX36
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1FE

Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MJSW0SA2\DR1MJSW0SA2\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MJSW0SA2\DR1MJSW0SA2
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MJSW0SI1010
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MJSW0SI1010\DR1MJSW0SI1010\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MJSW0SI1010\DR1MJSW0SI1010
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MJSW0SI1640
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MJSW0SI1640\DR1MJSW0SI1640\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MJSW0SI1640\DR1MJSW0SI1640
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\outpu

Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MRJO0SX14\DR1MRJO0SX14\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MRJO0SX14\DR1MRJO0SX14
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MRJO0SX194
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MRJO0SX194\DR1MRJO0SX194\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MRJO0SX194\DR1MRJO0SX194
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MRJO0SX284
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MRJO0SX284\DR1MRJO0SX284\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MRJO0SX284\DR1MRJO0SX284
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1M

Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MWBT0SI1553\DR1MWBT0SI1553\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MWBT0SI1553\DR1MWBT0SI1553
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MWBT0SI2183
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MWBT0SI2183\DR1MWBT0SI2183\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MWBT0SI2183\DR1MWBT0SI2183
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MWBT0SI923
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MWBT0SI923\DR1MWBT0SI923\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR1MWBT0SI923\DR1MWBT0SI923
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-mai

...

Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR7MTWH0SX290\DR7MTWH0SX290\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR7MTWH0SX290\DR7MTWH0SX290
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR7MTWH0SX380
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR7MTWH0SX380\DR7MTWH0SX380\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR7MTWH0SX380\DR7MTWH0SX380
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FCMH1SA1
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FCMH1SA1\DR8FCMH1SA1\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FCMH1SA1\DR8FCMH1SA1
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FCMH1SA

Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FMLD0SI822\DR8FMLD0SI822\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FMLD0SI822\DR8FMLD0SI822
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FMLD0SI925
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FMLD0SI925\DR8FMLD0SI925\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FMLD0SI925\DR8FMLD0SI925
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FMLD0SX115
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FMLD0SX115\DR8FMLD0SX115\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8FMLD0SX115\DR8FMLD0SX115
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\

Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MDAW1SX373\DR8MDAW1SX373\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MDAW1SX373\DR8MDAW1SX373
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJLN0SA1
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJLN0SA1\DR8MJLN0SA1\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJLN0SA1\DR8MJLN0SA1
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJLN0SA2
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJLN0SA2\DR8MJLN0SA2\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJLN0SA2\DR8MJLN0SA2
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJLN0SI1449
Combi

Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJTH0SX126\DR8MJTH0SX126\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJTH0SX126\DR8MJTH0SX126
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJTH0SX216
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJTH0SX216\DR8MJTH0SX216\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJTH0SX216\DR8MJTH0SX216
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJTH0SX306
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJTH0SX306\DR8MJTH0SX306\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MJTH0SX306\DR8MJTH0SX306
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\

Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MRES0SX47\DR8MRES0SX47
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MSLB0SA1
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MSLB0SA1\DR8MSLB0SA1\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MSLB0SA1\DR8MSLB0SA1
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MSLB0SA2
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MSLB0SA2\DR8MSLB0SA2\combined.txt
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MSLB0SA2\DR8MSLB0SA2
Processing directory: C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MSLB0SI1193
Combined transcript written to C:\Users\gdhir\Downloads\EchoMind-main\EchoMind-main\output\DR8MSLB0SI1193\DR8MSLB0SI1193\combined.txt
Proce